# Crawling Berita

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urlparse, urljoin

# Menambahkan header User-Agent adalah praktik yang baik
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
# Jumlah maksimal percobaan jika request gagal
MAX_RETRIES = 3
# Jumlah artikel yang diinginkan per kategori
ARTIKEL_PER_KATEGORI = 25

def dapatkan_kategori_berita():
    """
    Fungsi untuk mengambil daftar semua kategori berita dari menu navigasi
    website bangsaonline.com.
    """
    print("Mencari kategori berita di bangsaonline.com...")
    kategori_list = {}
    url_home = "https://bangsaonline.com/"
    try:
        response = requests.get(url_home, headers=HEADERS, timeout=15)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        
        nav_menu = soup.select_one('ul#nav')
        if not nav_menu:
            print("Menu navigasi (ul#nav) tidak ditemukan.")
            return {}

        for item in nav_menu.find_all("a"):
            href = item.get("href")
            nama_kategori = item.get_text(strip=True)
            
            if href and nama_kategori:
                path_parts = urlparse(href).path.strip("/").split("/")
                
                if len(path_parts) == 2 and path_parts[0] == 'kanal':
                    url_lengkap = urljoin(url_home, href)
                    if nama_kategori not in kategori_list:
                        kategori_list[nama_kategori] = url_lengkap

        print(f"Ditemukan {len(kategori_list)} kategori berita valid.")
        
    except requests.exceptions.RequestException as e:
        print(f"Gagal mengambil daftar kategori berita: {e}")
        
    return kategori_list

def scrape_semua_berita():
    """
    Fungsi utama untuk melakukan scraping berita dari semua kategori yang ditemukan,
    dengan implementasi pagination dan retry mechanism.
    """
    daftar_kategori = dapatkan_kategori_berita()

    if not daftar_kategori:
        print("Tidak ada kategori yang bisa di-scrape. Program berhenti.")
        return

    data_berita = []
    scraped_links = set()
    url_home = "https://bangsaonline.com/"

    for nama_kategori, url_kategori in daftar_kategori.items():
        print(f"\n--- Scraping Kategori: {nama_kategori.upper()} ---")
        artikel_diambil = 0
        halaman_ke = 1
        
        # Loop akan berjalan selama artikel yang diambil < target
        while artikel_diambil < ARTIKEL_PER_KATEGORI:
            
            # --- LOGIKA PAGINATION BARU ---
            if halaman_ke == 1:
                current_url = url_kategori
            else:
                current_url = f"{url_kategori}?page={halaman_ke}"
            
            print(f"   -> Mengambil halaman {halaman_ke}: {current_url}")
            
            response_kategori = None
            for attempt in range(MAX_RETRIES):
                try:
                    response_kategori = requests.get(current_url, headers=HEADERS, timeout=15)
                    response_kategori.raise_for_status()
                    break
                except requests.exceptions.RequestException as e:
                    print(f"      Gagal mengambil halaman kategori (percobaan {attempt + 1}/{MAX_RETRIES}): {e}")
                    time.sleep(2)
            
            if not response_kategori:
                print(f"   Gagal mengambil halaman kategori {current_url} setelah {MAX_RETRIES} percobaan. Lanjut ke kategori berikutnya.")
                break

            soup_kategori = BeautifulSoup(response_kategori.text, "html.parser")
            list_artikel = soup_kategori.select("h3.entry-title a")
            
            # --- KONDISI BERHENTI BARU ---
            # Jika halaman tidak ada artikelnya, hentikan untuk kategori ini
            if not list_artikel:
                print("   Tidak ada artikel di halaman ini, selesai untuk kategori ini.")
                break

            for artikel in list_artikel:
                if artikel_diambil >= ARTIKEL_PER_KATEGORI:
                    break
                
                link = urljoin(url_home, artikel.get("href", ""))
                if not link or link in scraped_links:
                    continue

                scraped_links.add(link)
                
                resp_detail = None
                for attempt in range(MAX_RETRIES):
                    try:
                        resp_detail = requests.get(link, headers=HEADERS, timeout=15)
                        resp_detail.raise_for_status()
                        break
                    except requests.exceptions.RequestException as e:
                        print(f"      Gagal mengambil detail dari {link} (percobaan {attempt + 1}/{MAX_RETRIES})")
                        time.sleep(1)

                if not resp_detail:
                    print(f"   -> Gagal total mengambil detail dari {link}. Melewatkan artikel ini.")
                    continue
                        
                soup_detail = BeautifulSoup(resp_detail.text, "html.parser")
                judul_element = soup_detail.select_one("h1.entry-title")
                konten_berita = soup_detail.select_one("div.post")
                
                if judul_element and konten_berita:
                    judul = judul_element.get_text(strip=True)
                    
                    id_berita = None
                    try:
                        path_parts = urlparse(link).path.strip("/").split("/")
                        if len(path_parts) > 1 and path_parts[1].isdigit():
                            id_berita = path_parts[1]
                    except (IndexError, AttributeError):
                        id_berita = None
                    
                    for unwanted in konten_berita.select("div.baca-juga, div.shared-icons"):
                        unwanted.decompose()
                    
                    paragraf = [p.get_text(strip=True) for p in konten_berita.select("p")]
                    isi = " ".join(paragraf)

                    if isi:
                        data_berita.append({
                            "id_berita": id_berita, "kategori": nama_kategori,
                            "judul": judul, "isi_berita": isi, "link": link
                        })
                        artikel_diambil += 1
                        print(f"({artikel_diambil}/{ARTIKEL_PER_KATEGORI}) Berhasil scrape: {judul[:60]}...")
                
                time.sleep(1) 

            # Pindah ke halaman berikutnya untuk iterasi selanjutnya
            halaman_ke += 1

    if not data_berita:
        print("\nTidak ada berita yang berhasil di-scrape.")
        return

    df = pd.DataFrame(data_berita)
    df = df[["id_berita", "kategori", "judul", "isi_berita", "link"]]
    
    nama_file = "hasil_scraping_berita_bangsaonline.csv"
    df.to_csv(nama_file, index=False, encoding="utf-8-sig")
    print(f"\n✅ Proses scraping selesai. {len(df)} berita disimpan ke '{nama_file}'")
    
    return df

# --- Untuk Menjalankan Seluruh Proses Scraping ---
if __name__ == "__main__":
    df_hasil = scrape_semua_berita()
    if df_hasil is not None:
        pd.set_option('display.max_colwidth', 100)
        print("\nContoh hasil data:")
        print(df_hasil.head())

Mencari kategori berita di bangsaonline.com...


Ditemukan 37 kategori berita valid.

--- Scraping Kategori: JATIM ---
   -> Mengambil halaman 1: https://bangsaonline.com/kanal/jawa-timur


(1/25) Berhasil scrape: Hasil Asesmen, Polda Jatim Tutup Sementara Area Ponpes Al Kh...


(2/25) Berhasil scrape: Jaga Ketertiban Warga, Polres Pasuruan Perketat Izin Karnava...


(3/25) Berhasil scrape: Pemkab Situbondo Genjot Kualitas Tembakau Lewat Pelatihan Pa...


...

(13/25) Berhasil scrape: Satreskrim Polres Mojokerto Tangkap 2 Pelaku Residivis Kasus...


(14/25) Berhasil scrape: Lagi, MBG di Tuban Diduga Jadi Penyebab 8 Siswa SMKN Tambakb...


(15/25) Berhasil scrape: Protes Kelalaian Trans7 Soal Ponpes Lirboyo, Cak Imin: KPI H...


(16/25) Berhasil scrape: Tim Gabungan Gelar Operasi Tertib Pajak Kendaraan Bermotor d...


KeyboardInterrupt: 

In [7]:
df_hasil.head()

,id_berita,kategori,judul,isi_berita,link
0,153055,Jatim,Ada Santri yang Diduga Dihukum untuk Cor Bangunan Musala Ambruk Ponpes Al Khoziny,"SIDOARJO,BANGSAONLINE.com- Proses pencarian korban para santri tertimbun reruntuhan musala ambru...",https://bangsaonline.com/berita/153055/ada-santri-yang-diduga-dihukum-untuk-cor-bangunan-musala-...
1,153054,Jatim,"Warga Dringu Lapor Dugaan Penipuan Kavling, Polres Probolinggo Kota Siap Tindaklanjuti","PROBOLINGGO, BANGSAONLINE.com- Polres Probolinggo Kota menerima laporan dari warga Desa/Kecamata...",https://bangsaonline.com/berita/153054/warga-dringu-lapor-dugaan-penipuan-kavling-polres-proboli...
2,153053,Jatim,"Hadiri Pengukuhan Pengurus Kormi Kota Kediri, Gus Qowim Dorong Olahraga untuk Semua Usia","KOTA KEDIRI, BANGSAONLINE.com- Wakil Wali Kota Kediri, Qowimuddin Thoha atau yang akrab disapa G...",https://bangsaonline.com/berita/153053/hadiri-pengukuhan-pengurus-kormi-kota-kediri-gus-qowim-do...
3,153052,Jatim,"Datang ke RS William Booth Perpanjang Rujukan Anak, Pria di Surabaya Malah Curi Motor","SURABAYA,BANGSAONLINE.com- Polsek Wonokromo menangkap pelaku pencurian motor di area parkir Rum...",https://bangsaonline.com/berita/153052/datang-ke-rs-william-booth-perpanjang-rujukan-anak-pria-d...
4,153050,Jatim,Polwan Polresta Sidoarjo Layani Keluarga Korban Musala Ambruk Ponpes Al Khoziny,"SIDOARJO,BANGSAONLINE.com- Polwan Polresta Sidoarjo memberikan pelayanan bagi keluarga korban tr...",https://bangsaonline.com/berita/153050/polwan-polresta-sidoarjo-layani-keluarga-korban-musala-am...
